# Sudoku ver. 3

### 문제
data = 0,0,0,0,0,0,0,5,0,7,0,9,6,0,2,0,8,0,3,0,5,0,7,0,6,1,9,8,0,0,2,0,0,3,0,0,0,0,0,1,3,5,0,0,0,0,0,3,0,0,8,0,0,6,5,3,4,0,8,0,2,0,1,0,8,0,3,0,1,5,0,4,0,6,0,0,0,0,0,0,0

### 원리
빈 칸이 없을 때까지 반복 > 9번 반복 > 빈 칸 개수만큼 반복<br>
{
1) 한 칸 남은 행, 열, 구역 있는지 조사 -> 있으면 채우기<br>
1-1) 불확실성 리스트에 1)에서 채워진 칸 있으면 해당 칸을 1)에서 채워진 숫자로 대체<br>
2) 특정 숫자 해당하는 지역, 칸, 열 다 지움<br>
3) 해당 지역에 가능한 숫자가 그 숫자밖에 없다면 (ex)지역에 자리 하나 남음, 열에 하나 남음, 행에 하나 남음) 채우기<br>
3-1) 불확실성 리스트에 3)에서 채워진 칸 있으면 해당 칸을 3)에서 채워진 숫자로 대체<br>
4) 채워지면 2)로 돌아가 해당 숫자 재검증하기, 3)의 경우 발생하면 같은 과정 반복<br>
5) 더이상 채워지는 칸이 없으면 불확실성 리스트에 해당 숫자 채워넣기
}<br><br>
만약 두 번째 반복문의 결과가 3번 이상 같다면, 불확실성 리스트 검증<br>

위 과정 코드가 완성될 때까지 반복

### 수정 사항
1) updatemain함수 만들어서 본 코드 간소화 <br>
2) 오타 수정

### 결과
- 난이도 제일 높던 것이 풀림...야호
- 빈칸 많은 난이도 중은 끝까지 안풀림ㅜ

case1 = 2,0,0,9,0,1,4,0,0,0,7,0,8,0,0,0,0,2,0,0,0,0,0,0,9,0,0,0,2,0,3,6,0,0,0,0,8,3,0,0,4,0,0,1,9,0,0,0,0,9,8,0,2,0,0,0,8,0,0,0,0,0,0,1,0,0,0,0,6,0,5,0,0,0,3,2,0,9,0,0,8

답: https://post.naver.com/viewer/postView.naver?volumeNo=32219313&memberNo=512601&navigationType=push

case2 = 0,0,0,0,0,1,0,9,0,0,5,0,0,3,0,0,0,7,3,0,6,0,0,2,0,8,0,0,9,0,6,0,0,3,0,0,5,0,3,0,0,0,0,0,0,0,6,0,0,8,0,0,4,0,0,0,0,3,0,9,0,7,0,0,0,2,0,0,0,9,0,0,0,0,0,7,0,4,0,0,0

답: https://post.naver.com/viewer/postView.nhn?volumeNo=28679719&memberNo=512601

## 1. 함수 정의

### 데이터 처리 관련 함수

In [1]:
import numpy as np

def preProcessing(problem):
    # 문자열 데이터를 받아 9x9의 2차원 np array로 반환
    data = problem.split(',')
    frame = np.reshape(data,(9,9)).astype('int')
    return frame

def splitFrame(frame):
    '''
    9x9의 2차원 int형 array를 받아, dictionary 두 개를 반환
    
    1) num_area
    - key: 1 2 3 / 4 5 6 / 7 8 9 (나눠진 지역의 위치)
    - value: 해당 위치의 3x3 지역의 값들로 이루어진 2차원 array
      [:3,:3]  [:3,3:6]  [:3,6:9]
      [3:6,:3] [3:6,3:6] [3:6,6:9]
      [6:9,:3] [6:9,3:6] [6:9,6:9]
      
    2) num_index
    - key: 1 2 3 / 4 5 6 / 7 8 9 (나눠진 지역의 위치)
    - value: 해당 위치의 3x3 지역의 값들의 인덱스로 이루어진 list in list
      [[r1,r2,r3],[c1,c2,c3]] * 9
    '''
    num_area={}
    num_index={}
    for i in range(3):
        r=(i+1)*3
        for j in range(3):
            c=(j+1)*3
            key=j+i*3+1
            num_area[key]=frame[i*3:r,j*3:c]
            num_index[key]=[list(range(i*3,r)),list(range(j*3,c))]
    return num_area, num_index

def printFrame(frame):
    # 스도쿠 그려줌
    for i in range(9):
        for j in range(9):
            if j==8:
                print(frame[i][j],'| ')
            elif (j+1)%3==0:
                print(frame[i][j],'| ',end='')
            else:
                print(frame[i][j],' ',end='')
        if (i+1)%3==0:
            print('- - - - - - - - - - - - - - -')
            
def nparrToList(arr):
    '''
    n*2 크기의 np array를 n*2크기의 list로 바꿔줌
    
    parameter: arr -> numpy.ndarray (n * 2 크기의 2차원 np array)
    
    return: new_list -> list (n * 2 크기의 list in list)
    '''
    new_list=[]
    for x in arr:
        new_list.append([x[0],x[1]])
    return new_list

### 특정 값을 찾아주는 함수

In [2]:
def findArea(frame,r_index,c_index):
    '''
    row와 column의 index 값을 넣어 주면 어느 area에 있는지 번호를 찾아 int로 반환함
    
    parameter: frame -> numpy.ndarray (9 * 9 크기의 2차원 np array) 
               r_index, c_index -> int (행, 열 인덱스 값)
               
    return: i -> int (area 번호)
    
    '''
    Num_Area, Num_Index = splitFrame(frame)
    for i in range(1,10):
        area=Num_Index[i]
        r=area[0]
        c=area[1]
        if r_index in r and c_index in c:
            return i
            
def findAreaIndex(frame,a):
    '''
    frame과 area를 넣으면 해당 area의 index 목록을 list로 반환함
    
    parameter: frame -> numpy.ndarray (9 * 9 크기의 2차원 np array) 
               a -> int (area 번호)
               
    return: index -> list 
            ex) [[r1,c1],[r2,c2],...]
    '''
    num_area, num_index=splitFrame(frame)
    area=num_index[a]
    r=area[0]
    c=area[1]
    index=[]
    for i in range(3):
        for j in range(3):
            index.append([r[i],c[j]])
    return index

def findEmptyIndex(frame):
    '''
    frame을 넣어주면 값을 채워야 할 행, 열의 인덱스 목록(0인 목록)을 np array로 반환함
    
    parameter: frame -> numpy.ndarray (9 * 9 크기의 2차원 np array) 
    
    return: numpy.ndarray
            [[r1,c1],[r2,c2],...]
    '''
    empty_list=[]
    row, col = np.where(frame==0)
    for i in range(len(row)):
        empty_list.append([row[i],col[i]])
    return np.array(empty_list)

def findNone(array):
    '''
    행, 열 구역에서 없는 숫자를 찾아 숫자들의 리스트를 반환함
    만일 숫자가 하나라면, 정수 하나를 반환함
    
    parameter: array -> numpy.ndarray (1 * 9 (행, 열 array) or 3 * 3 (구역 array))
    
    return: num -> int (요소가 1개일 경우) or list (1 * n)
    '''
    num=list(range(1,10))
    if len(array)==9:
        for i in range(9):
            if array[i] in num:
                num.remove(array[i])
    else:
        for i in range(3):
            for j in range(3):
                if array[i][j] in num:
                    num.remove(array[i][j])
    if len(num)==1:
        return num[0]
    return num

### 세어주는 함수

In [3]:
def countEmpty(r,c,area):
    # 행, 열, 구역의 빈 칸의 개수를 각각 반환함
    r_=np.where(r==0) # 튜플이구나,,,
    c_=np.where(c==0)
    r__,c__=np.where(area==0)
    return len(r_[0]), len(c_[0]), len(r__)

def countNum(frame,num):
    '''
    frame과 num을 입력하면 현재 스도쿠 퍼즐에서 num이 채워진 칸의 개수를 반환함
    
    parameter: frame -> numpy.ndarray (9 * 9 크기의 2차원 np array) 
               num -> int (스도쿠에서 찾고 싶은 번호(1 ~ 9))
               
    return: count -> int (num이 채워진 칸의 개수)
    '''
    count=0
    for i in range(9):
        for j in range(9):
            if frame[i][j]==num:
                count+=1
    return count

### 제거해주는 함수

In [4]:
def delEmpty(empty_list,l):
    '''
    l에 포함되는 인덱스를 제거한 empty_list를 np array로 반환함
    l은 제거할 row, column index의 쌍으로 이루어진 list in list -> [[r1,c1],[r2,c2],...]
    
    parameter: empty_list -> numpy.ndarray (n * 2 크기의 2차원 np array) 
               l -> list (n * 2 크기의 list in list)
               
    return: new_list -> numpy.ndarray (n * 2 크기의 2차원 np array) 
    '''
    new_list=nparrToList(empty_list) # 사용자 정의 함수 이용
    for x in l:
        new_list.remove(x)
    return np.array(new_list)
    
def delSquares(frame,r,c,area):
    '''
    frame에서 r(행), c(열), area(구역)에 해당하는 부분들의 값을 100으로 바꾼 후 반환
    후에 해당하는 칸을 제외시키고 싶을 때 써먹을 수 있음
    
    parameter: frame -> numpy.ndarray (9 * 9 크기의 2차원 np array) 
               r, c -> list (1 * n 크기의 행, 열 인덱스를 담은 list)
               area -> list (1 * n 크기의 구역 번호를 담은 list)
               
    return: new_frame -> numpy.ndarray (9 * 9 크기의 2차원 np array) 
    '''
    new_frame=frame.copy()
    for i in range(len(r)):
        new_frame[r[i]]=100
    for i in range(len(c)):
        new_frame[:,c[i]]=100
    for n in area:
        index=findAreaIndex(frame,n)
        for x in index:
            new_frame[x[0]][x[1]]=100
    return new_frame

### 복합적인 함수

In [5]:
def appendDelList(value,i,r,c,row,column,num_index,del_index):
    '''
    제외할 항목을 담은 리스트들(row, column, num_index, del_index)에서 제외할 항목인 [r][c]에서의 값을 추가해줌
    ** 입력받은 파라미터의 값을 직접 바꾸므로 return값 없음, 이용시 주의 **
    
    parameter: num, i -> int (업데이트된 값, 반복문 시행,,, 복잡하다 복잡해)
               r, c -> int (행, 열 인덱스 값)
               row, column, num_index, del_index -> list
    
    return: 없음
    '''
    del_index.append([r,c])
    if (value==i+1):
        row.append(r)
        column.append(c)
        num_index+=findAreaIndex(frame,findArea(frame,r,c))
    
def isSame(diff, frame=0):
    '''
    두 frame으로 이루어진 np array diff를 받아, 새 frame으로 update 한 후 두 frame이 같은지 판별

    parameter: diff -> numpy.ndarray (9 * 9 크기의 2차원 np array 2개를 성분으로 가진 3차원 np array)
               frame = 0 -> numpy.ndarray (9 * 9 크기의 2차원 np array) 
               파라미터가 없는 경우 diff의 두 frame 비교
    
    return: True(bool) ; 두 frame이 같은 경우
    '''
    if type(frame)!=int:
        diff[0]=diff[1]
        diff[1]=frame
    
    compare=(diff[0]==diff[1])
    a,b=np.where(compare==False)
    if (len(a)==0):
        return True

def updateFrame(frame, virtual_frame, r, c, value):
    '''
    특정 값을 업데이트해주는 전 과정을 수행한 후, 업데이트된 frame과 virtual_frame을 return
    
    parameter: frame -> numpy.ndarray (9 * 9 크기의 2차원 np array (int)) 
               virtual_frame -> numpy.ndarray (9 * 9 크기의 2차원 np array (str))
               r, c, value -> int (r, c: 행, 열 인덱스 값 / value: 업데이트할 값)
               
    return: frame -> numpy.ndarray (9 * 9 크기의 2차원 np array (int)) 
            virtual_frame -> numpy.ndarray (9 * 9 크기의 2차원 np array (str))
    '''

    frame[r][c]=value
    area=findArea(frame,r,c) # 사용자 정의 함수 이용
    
    for i in range(9):
        for j in range(9):
            index=virtual_frame[i][j].find(str(value))
            include=(i==r or j==c or findArea(frame,i,j)==area) # 해당 인덱스 값이랑 같을 경우
            if index!=-1 and index!=0 and include:
                virtual_frame[i][j]=virtual_frame[i][j].replace(virtual_frame[i][j][index],'') # virtual_frame에서 해당 값 제거해줌
    virtual_frame[r][c]=value
    
    return frame, virtual_frame

### 스도쿠 풀어주는 함수

사용자 정의 함수 많이 이용하므로 순서 바꾸지 말 것.,

In [6]:
def updateByFrame(frame,empty_list,diff):
    '''
    parameter: frame -> numpy.ndarray (9 * 9 크기의 2차원 np array (int)) 
               empty_list -> numpy.ndarray (n * 2 크기의 2차원 np array (int))
               diff -> numpy.ndarray (9 * 9 크기의 2차원 np array 2개를 성분으로 가진 3차원 np array)
     
    return: frame -> numpy.ndarray (9 * 9 크기의 2차원 np array (int)) 
            virtual_frame -> numpy.ndarray (9 * 9 크기의 2차원 np array (str))
            empty_list -> numpy.ndarray (n * 2 크기의 2차원 np array (int))
    
    frame, virtual_frame, empty_list를 업데이트하여 반환함
    '''
    virtual_frame=np.copy(frame).astype(str) # virtual frame 수저
    while(1):
        for i in range(9):
            row, column=list(np.where(frame==i+1))
            row=list(row) 
            column=list(column)
            area=[]
            num_index=[]
            for j in range(len(row)):
                area.append(findArea(frame,row[j],column[j]))
                num_index+=findAreaIndex(frame,area[j])

            empty_len=[100,len(empty_list)]
            now=1

            while(empty_len[now]<empty_len[now-1]): 
                del_index=[]
                for k in empty_list:
                    r=k[0]
                    c=k[1]
                    n=findArea(frame,r,c)
                    e_r,e_c,e_a=countEmpty(frame[r],frame[:,c],Num_Area[n])
                    if e_r==1:
                        value=findNone(frame[r])
                        frame, virtual_frame=updateFrame(frame, virtual_frame, r, c, value) 
                        appendDelList(value,i,r,c,row,column,num_index,del_index)
                    elif e_c==1:
                        value=findNone(frame[:,c])
                        frame, virtual_frame=updateFrame(frame, virtual_frame, r, c, value) 
                        appendDelList(value,i,r,c,row,column,num_index,del_index)
                    elif e_a==1:
                        value=findNone(Num_Area[n])
                        frame, virtual_frame=updateFrame(frame, virtual_frame, r, c, value) 
                        appendDelList(value,i,r,c,row,column,num_index,del_index)
                    elif (r not in row) and (c not in column) and ([r,c] not in num_index):
                        temp_frame=delSquares(frame,row,column,area)
                        Temp_Area, Temp_Index=splitFrame(temp_frame)
                        e_r,e_c,e_a=countEmpty(temp_frame[r],temp_frame[:,c],Temp_Area[n])
                        if e_r==1 or e_c==1 or e_a==1:
                            frame, virtual_frame=updateFrame(frame, virtual_frame, r, c, i+1) 
                            appendDelList(i+1,i,r,c,row,column,num_index,del_index)

                empty_list=delEmpty(empty_list,del_index)
                empty_len.append(len(empty_list))
                now+=1

            temp_frame=delSquares(frame,row,column,area) 
            remain_index=list(findEmptyIndex(temp_frame))
            if remain_index!=0:
                for k in range(len(remain_index)):
                    r=remain_index[k][0]
                    c=remain_index[k][1]
                    if str(i+1) not in virtual_frame[r][c]:
                        virtual_frame[r][c]+=str(i+1)
        

        if isSame(diff, virtual_frame):
            break
            
    return frame, virtual_frame, empty_list

def updateByBirtual(frame,virtual_frame,empty_list):
    '''
    parameter: frame -> numpy.ndarray (9 * 9 크기의 2차원 np array (int)) 
               virtual_frame -> numpy.ndarray (9 * 9 크기의 2차원 np array (str))
               empty_list -> numpy.ndarray (n * 2 크기의 2차원 np array (int))
               
    return: frame -> numpy.ndarray (9 * 9 크기의 2차원 np array (int)) 
            virtual_frame -> numpy.ndarray (9 * 9 크기의 2차원 np array (str))
            empty_list -> numpy.ndarray (n * 2 크기의 2차원 np array (int))
    
    virtual_frame에서 각 칸의 문자열을 조사하여 업데이트된 frame, virtual_frame, empty_list를 반환함
    while(1), frame동안 같을 경우 break
    1) 문자열 길이가 2 -> a[1]이 해당 칸의 숫자이므로 해당 숫자 업데이트
    2) 문자열 길이가 3이고 두개 중복 -> 나머지 칸에서 해당 숫자 빼줌
    '''
    
    diff=np.array([virtual_frame,virtual_frame])
    new_empty_list=nparrToList(empty_list)
    
    while(1):
        for i in new_empty_list:
            r=i[0]
            c=i[1]
            
            if len(virtual_frame[r][c])==2:
                value=int(virtual_frame[r][c][1])
                frame, virtual_frame=updateFrame(frame, virtual_frame, r, c, value)
                new_empty_list.remove(i)

        for i in range(9):
            temp_by_r=[]
            temp_by_c=[]
            temp_by_a=[]
            for j in new_empty_list:
                r=j[0]
                c=j[1]
                if r==i:
                    temp_by_r.append(c)
                if c==i:
                    temp_by_c.append(r)
                if findArea(frame,r,c)==(i+1):
                    if j in findAreaIndex(frame,i+1):
                        temp_by_a.append(j)
            # 행
            v_r=virtual_frame[i]
            if (len(temp_by_r)>1):
                for k in temp_by_r:
                    if len(v_r[k])==3:
                        a=np.where(v_r==v_r[k])
                        if len(a[0])==2:
                            c1=v_r[k][1]
                            c2=v_r[k][2]
                            for l in temp_by_r:
                                if l not in a[0]:
                                    if c1 in v_r[l]:
                                        virtual_frame[i][l]=virtual_frame[i][l].replace(c1,'')
                                    if c2 in v_r[l]:
                                        virtual_frame[i][l]=virtual_frame[i][l].replace(c2,'')

            # 열
            v_c=virtual_frame[:,i]
            if (len(temp_by_c)>1):
                for k in temp_by_c:
                    if len(v_c[k])==3:
                        a=np.where(v_c==v_c[k])
                        if len(a[0])==2:
                            c1=v_c[k][1]
                            c2=v_c[k][2]
                            for l in temp_by_c:
                                if l not in a[0]:
                                    if c1 in v_c[l]:
                                        virtual_frame[l][i]=virtual_frame[l][i].replace(c1,'')
                                    if c2 in v_c[l]:
                                        virtual_frame[l][i]=virtual_frame[l][i].replace(c2,'')

            # 구역
            v_a=findAreaIndex(frame,i+1)
            if (len(temp_by_a)>1):
                for k in temp_by_a:
                    r=k[0]
                    c=k[1]
                    if len(virtual_frame[r][c])==3:
                        a_r,a_c=np.where(virtual_frame==virtual_frame[r][c])
                        a=[]
                        for i in range(len(a_r)):
                            if [a_r[i],a_c[i]] in v_a:
                                a.append([a_r[i],a_c[i]])
                        if len(a)==2:
                            c1=virtual_frame[r][c][1]
                            c2=virtual_frame[r][c][2]
                            for l in temp_by_a:
                                r_=l[0]
                                c_=l[1]
                                if l not in a:
                                    if c1 in virtual_frame[r_][c_]:
                                        virtual_frame[r_][c_]=virtual_frame[r_][c_].replace(c1,'')
                                    if c2 in virtual_frame[r_][c_]:
                                        virtual_frame[r_][c_]=virtual_frame[r_][c_].replace(c2,'')
                                        
        if isSame(diff, virtual_frame):
            break
    
    return frame, virtual_frame, np.array(new_empty_list)

## 2. 코드

### 1) case 1

In [7]:
import numpy as np

level=0
while(1):
    level+=1
    print('\n',"level.",level)
    print("         ** 9x9 스도쿠 **")
    print(" -----------------------------------")
    print("| - 스도쿠의 1~9열을 순서대로 입력  |")
    print("| - 빈 공간은 0으로 설정            |")
    print("| - 구분은 콤마(,):                 |")
    print(" -----------------------------------")

    while(1):
        data=str(input())
        if len(data)==81*2-1:
            break;
        else:
            print("숫자가 {}개입니다. 81개를 입력하세요".format(int(len(data)/2+0.5)))

    # 데이터 정리
    frame=preProcessing(data)
    Num_Area, Num_Index=splitFrame(frame) # 3*3 지역

    # 빈 칸의 인덱스
    # empty_list[:,0] = r, empty_list[:,1] = c
    empty_list=findEmptyIndex(frame)

    # frame 비교용
    diff=np.array([frame,frame])

    print('')
    print('problem: ')
    printFrame(frame)
    print('')

    same=0
    while(len(empty_list)>0):
        frame, virtual_frame, empty_list=updateByFrame(frame,empty_list,diff)
        frame1=np.copy(frame)
        frame, virtual_frame, empty_list=updateByBirtual(frame,virtual_frame,empty_list)
        frame2=np.copy(frame)
        frames=np.array([frame1,frame2])
        if isSame(frames):
            same+=1
        if same>1:
            print("어려워서 다 못 풀었어요 ㅠ",'\n')
            break

    print('answer:')
    printFrame(frame)


 level. 1
         ** 9x9 스도쿠 **
 -----------------------------------
| - 스도쿠의 1~9열을 순서대로 입력  |
| - 빈 공간은 0으로 설정            |
| - 구분은 콤마(,):                 |
 -----------------------------------


KeyboardInterrupt: Interrupted by user

### 2) case 2

In [ ]:
import numpy as np

print("         ** 9x9 스도쿠 **")
print(" -----------------------------------")
print("| - 스도쿠의 1~9열을 순서대로 입력  |")
print("| - 빈 공간은 0으로 설정            |")
print("| - 구분은 콤마(,):                 |")
print(" -----------------------------------")

while(1):
    data=str(input())
    if len(data)==81*2-1:
        break;
    else:
        print("숫자가 {}개입니다. 81개를 입력하세요".format(int(len(data)/2+0.5)))

# 데이터 정리
frame=preProcessing(data)
Num_Area, Num_Index=splitFrame(frame) # 3*3 지역
    
# 빈 칸의 인덱스
# empty_list[:,0] = r, empty_list[:,1] = c
empty_list=findEmptyIndex(frame)

# frame 비교용
diff=np.array([frame,frame])

print('')
print('problem: ')
printFrame(frame)
print('')

same=0
while(len(empty_list)>0):
    frame, virtual_frame, empty_list=updateByFrame(frame,empty_list,diff)
    frame1=np.copy(frame)
    frame, virtual_frame, empty_list=updateByBirtual(frame,virtual_frame,empty_list)
    frame2=np.copy(frame)
    frames=np.array([frame1,frame2])
    if isSame(frames):
        same+=1
    if same>1:
        print("어려워서 다 못 풀었어요 ㅠ",'\n')
        break
    
print('answer:')
printFrame(frame)

### 3) 교수님 문제

In [ ]:
import numpy as np

print("         ** 9x9 스도쿠 **")
print(" -----------------------------------")
print("| - 스도쿠의 1~9열을 순서대로 입력  |")
print("| - 빈 공간은 0으로 설정            |")
print("| - 구분은 콤마(,):                 |")
print(" -----------------------------------")

while(1):
    data=str(input())
    if len(data)==81*2-1:
        break;
    else:
        print("숫자가 {}개입니다. 81개를 입력하세요".format(int(len(data)/2+0.5)))

# 데이터 정리
frame=preProcessing(data)
Num_Area, Num_Index=splitFrame(frame) # 3*3 지역
    
# 빈 칸의 인덱스
# empty_list[:,0] = r, empty_list[:,1] = c
empty_list=findEmptyIndex(frame)

# frame 비교용
diff=np.array([frame,frame])

print('')
print('problem: ')
printFrame(frame)
print('')

same=0
while(len(empty_list)>0):
    frame, virtual_frame, empty_list=updateByFrame(frame,empty_list,diff)
    frame1=np.copy(frame)
    frame, virtual_frame, empty_list=updateByBirtual(frame,virtual_frame,empty_list)
    frame2=np.copy(frame)
    frames=np.array([frame1,frame2])
    if isSame(frames):
        same+=1
    if same>1:
        print("어려워서 다 못 풀었어요 ㅠ",'\n')
        break
    
print('answer:')
printFrame(frame)

In [8]:
import numpy as np

print("         ** 9x9 스도쿠 **")
print(" -----------------------------------")
print("| - 스도쿠의 1~9열을 순서대로 입력  |")
print("| - 빈 공간은 0으로 설정            |")
print("| - 구분은 콤마(,):                 |")
print(" -----------------------------------")

while(1):
    data=str(input())
    if len(data)==81*2-1:
        break;
    else:
        print("숫자가 {}개입니다. 81개를 입력하세요".format(int(len(data)/2+0.5)))

# 데이터 정리
frame=preProcessing(data)
Num_Area, Num_Index=splitFrame(frame) # 3*3 지역
    
# 빈 칸의 인덱스
# empty_list[:,0] = r, empty_list[:,1] = c
empty_list=findEmptyIndex(frame)

# frame 비교용
diff=np.array([frame,frame])

print('')
print('problem: ')
printFrame(frame)
print('')

same=0
while(len(empty_list)>0):
    frame, virtual_frame, empty_list=updateByFrame(frame,empty_list,diff)
    frame1=np.copy(frame)
    frame, virtual_frame, empty_list=updateByBirtual(frame,virtual_frame,empty_list)
    frame2=np.copy(frame)
    frames=np.array([frame1,frame2])
    if isSame(frames):
        same+=1
    if same>1:
        print("어려워서 다 못 풀었어요 ㅠ",'\n')
        break
    
print('answer:')
printFrame(frame)

         ** 9x9 스도쿠 **
 -----------------------------------
| - 스도쿠의 1~9열을 순서대로 입력  |
| - 빈 공간은 0으로 설정            |
| - 구분은 콤마(,):                 |
 -----------------------------------
0,0,0,0,0,0,0,5,0,7,0,9,6,0,2,0,8,0,3,0,5,0,7,0,6,1,9,8,0,0,2,0,0,3,0,0,0,0,0,1,3,5,0,0,0,0,0,3,0,0,8,0,0,6,5,3,4,0,8,0,2,0,1,0,8,0,3,0,1,5,0,4,0,6,0,0,0,0,0,0,0

problem: 
0  0  0 | 0  0  0 | 0  5  0 | 
7  0  9 | 6  0  2 | 0  8  0 | 
3  0  5 | 0  7  0 | 6  1  9 | 
- - - - - - - - - - - - - - -
8  0  0 | 2  0  0 | 3  0  0 | 
0  0  0 | 1  3  5 | 0  0  0 | 
0  0  3 | 0  0  8 | 0  0  6 | 
- - - - - - - - - - - - - - -
5  3  4 | 0  8  0 | 2  0  1 | 
0  8  0 | 3  0  1 | 5  0  4 | 
0  6  0 | 0  0  0 | 0  0  0 | 
- - - - - - - - - - - - - - -

answer:
6  4  8 | 9  1  3 | 7  5  2 | 
7  1  9 | 6  5  2 | 4  8  3 | 
3  2  5 | 8  7  4 | 6  1  9 | 
- - - - - - - - - - - - - - -
8  9  1 | 2  6  7 | 3  4  5 | 
4  7  6 | 1  3  5 | 9  2  8 | 
2  5  3 | 4  9  8 | 1  7  6 | 
- - - - - - - - - - - - - - -
5  3  4 | 7  8  6